In [19]:

from fastapi import HTTPException
from typing import Any, Dict, Optional
import json
import hmac
import hashlib
import httpx

AGENT_SECRET = 'dEcartes2026'

def json_dumps(payload: dict) -> bytes:
    return json.dumps(payload, separators=(",", ":"), sort_keys=True).encode()


def call_agent_hmac(ip, path, secret=AGENT_SECRET, json_body=None, method="POST", timeout=5):
    url = f"http://{ip}:8000{path}"
    
    if method == "DELETE":
        # Use path param (fqdn) as body for HMAC
        fqdn = path.split("/")[-1]
        body_bytes = fqdn.encode()
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers = {
            "Content-Type": "text/plain",
            "X-Signature": signature,
        }
        res = httpx.request("DELETE", url, content=body_bytes, headers=headers, timeout=timeout)

    else:
        headers = {"Content-Type": "application/json"}
        body_bytes = json_dumps(json_body) if json_body else b"{}"
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers["X-Signature"] = signature
        
        res = httpx.post(url, content=body_bytes, headers=headers, timeout=timeout)

    try:
        res.raise_for_status()
    except httpx.HTTPStatusError as e:
        raise HTTPException(
            status_code=e.response.status_code,
            detail=f"Agent error: {e.response.text}"
        )
    return res


In [23]:
# Load JSON from file
with open("../json/dnsproof.org.json", "r") as f:
    payload = json.load(f)

# Send request
response = call_agent_hmac(
    ip="136.115.36.6",
    path="/internal/dns/push",
    json_body=payload
)

print(response.json())

{'status': 'success'}
